In [ ]:
# sum 
# 225.0722880973707,



In [10]:
import numpy as np
import pandas as pd
import xgboost as xgb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import os

# ==========================================
# 1. 数据加载与预处理工具
# ==========================================

def load_full_data(npz_file):
    """
    加载完整的 npz 数据，返回包含所有 params 的 DataFrame。
    """
    print(f"Loading: {npz_file}")
    data = np.load(npz_file, allow_pickle=True)
    # 使用 param_names 作为列名构建完整 DataFrame
    df = pd.DataFrame(data["params"], columns=data["param_names"])
    return df

def prepare_data(sig_file, bkg_file, test_files_dict, features, test_size=0.2, seed=42):
    """
    准备训练、验证和测试数据。
    同时返回用于保存 CSV 的原始 DataFrame (仅 Val 和 Test，不含 Train)。
    """
    # 1. 加载完整数据
    df_sig = load_full_data(sig_file)
    df_bkg = load_full_data(bkg_file)
    
    # 检查特征是否存在
    for f in features:
        if f not in df_sig.columns:
            raise ValueError(f"Feature {f} missing in signal file")
        if f not in df_bkg.columns:
            raise ValueError(f"Feature {f} missing in background file")

    # 创建 Label
    y_sig = np.ones(len(df_sig))
    y_bkg = np.zeros(len(df_bkg))
    
    # 合并 DataFrame 和 Label
    df_all = pd.concat([df_sig, df_bkg], axis=0).reset_index(drop=True)
    y_all = np.concatenate([y_sig, y_bkg], axis=0)
    
    # 提取训练所需的特征矩阵 X
    X_all = df_all[features].values
    
    # 划分 Train / Val
    # 注意：这里我们同时划分 X(特征), y(标签) 和 df_all(原始完整数据)
    # random_state 保证了 X 和 df 的划分索引是一一对应的
    X_train, X_val, y_train, y_val, _, df_val = train_test_split(
        X_all, y_all, df_all, test_size=test_size, random_state=seed, stratify=y_all
    )
    
    # 2. 加载外部测试数据
    # test_data 结构: { 'name': (X_matrix, full_DataFrame) }
    test_data = {}
    for name, path in test_files_dict.items():
        df_temp = load_full_data(path)
        X_temp = df_temp[features].values
        test_data[name] = (X_temp, df_temp)
        
    return X_train, y_train, X_val, y_val, df_val, test_data

def save_to_csv(df_original, scores, out_dir, filename):
    """
    将原始数据与模型评分合并并保存为 CSV
    """
    os.makedirs(out_dir, exist_ok=True)
    save_path = os.path.join(out_dir, filename)
    
    # 避免修改原始内存中的 df，创建副本
    df_save = df_original.copy()
    
    # 添加模型分数
    df_save['model_score'] = scores
    
    # 保存
    print(f"Saving CSV: {save_path}")
    df_save.to_csv(save_path, index=False, float_format='%.6g')

# ==========================================
# 2. 绘图函数 (保持不变)
# ==========================================
def plot_score_distributions(val_dict, test_dict, out_dir, model_name):
    """
    val_dict: {'sig': scores, 'bkg': scores}
    test_dict: {'EF': scores, 'QF': scores, ...}
    """
    os.makedirs(out_dir, exist_ok=True)
    
    bins = np.linspace(0, 1, 21) # 稍微增加一点bin数使其更平滑
    bin_centers = 0.5 * (bins[:-1] + bins[1:])
    
    hist_val_bkg, _ = np.histogram(val_dict['bkg'], bins=bins, density=True)
    hist_base = hist_val_bkg.copy()
    hist_base[hist_base == 0] = np.nan
    
    fig, (ax_main, ax_ratio) = plt.subplots(2, 1, figsize=(10, 8), sharex=True, 
                                            gridspec_kw={'height_ratios': [3, 1], 'hspace': 0.05})
    
    colors = ['tab:blue', 'tab:green', 'tab:orange', 'tab:purple', 'tab:red']
    
    # --- PDF ---
    ax_main.hist(val_dict['sig'], bins=bins, density=True, histtype='step', 
                 color='red', linestyle='--', linewidth=1.5, label='Val Sig PDF')
    
    ax_main.hist(bins[:-1], bins, weights=hist_val_bkg, histtype='step', 
                 color='black', linestyle='--', linewidth=1.5, label='Val Bkg PDF')
    
    for i, (name, scores) in enumerate(test_dict.items()):
        color = colors[i % len(colors)]
        hist_curr, _ = np.histogram(scores, bins=bins, density=True)
        ax_main.hist(bins[:-1], bins, weights=hist_curr, histtype='step', 
                     alpha=0.3, color=color, label=f"{name} (PDF)")
        ax_main.hist(bins[:-1], bins, weights=hist_curr, histtype='step', 
                     linewidth=1, color=color)
        
        ratio = hist_curr / hist_base
        ax_ratio.step(bin_centers, ratio, where='mid', color=color, linewidth=1.5)

    # --- CDF ---
    ax_cdf = ax_main.twinx()
    sig_sorted = np.sort(val_dict['sig'])
    y_sig = np.arange(1, len(sig_sorted) + 1) / len(sig_sorted)
    ax_cdf.plot(sig_sorted, y_sig, color='red', linestyle='--', alpha=0.3, linewidth=2, label='Val Sig CDF')

    bkg_sorted = np.sort(val_dict['bkg'])
    y_bkg = 1.0 - np.arange(1, len(bkg_sorted) + 1) / len(bkg_sorted)
    ax_cdf.plot(bkg_sorted, y_bkg, color='black', linestyle='--', alpha=0.3, linewidth=2, label='Val Bkg Surv')

    ax_main.set_ylabel("Density")
    # ax_main.set_yscale('log') # Log scale 经常更好看
    ax_main.set_title(f"{model_name} Score Distributions")
    ax_main.legend(loc='upper center', ncol=2, frameon=False, fontsize=10)
    ax_main.grid(alpha=0.3)
    
    ax_cdf.set_ylabel("Probability")
    ax_cdf.set_ylim(0, 1.05)
    
    ax_ratio.axhline(1, color='black', linestyle='--', alpha=0.5) 
    ax_ratio.set_ylabel("Ratio / Val Bkg")
    ax_ratio.set_xlabel("Model Score")
    ax_ratio.set_ylim(0, 3) 
    ax_ratio.grid(alpha=0.3)
    
    save_path = os.path.join(out_dir, f"{model_name}_dist.png")
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Plot saved to {save_path}")

# ==========================================
# 3. 模型定义与运行函数
# ==========================================

# --- XGBoost ---
def run_xgboost(X_train, y_train, X_val, y_val, df_val, test_data, out_dir):
    print("\n=== Training XGBoost ===")
    
    clf = xgb.XGBClassifier(
        n_estimators=200, 
        learning_rate=0.1, 
        max_depth=5, 
        eval_metric="logloss",
        use_label_encoder=False,
        tree_method='hist',
        device='cpu'
    )
    
    clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    
    # --- 验证集预测与保存 ---
    val_probs = clf.predict_proba(X_val)[:, 1]
    
    # 保存 Validation CSV
    save_to_csv(df_val, val_probs, out_dir, "validation_set_xgb.csv")
    
    val_dict = {
        'sig': val_probs[y_val == 1],
        'bkg': val_probs[y_val == 0]
    }
    
    # --- 测试集预测与保存 ---
    test_dict = {}
    for name, (X_test, df_test) in test_data.items():
        probs = clf.predict_proba(X_test)[:, 1]
        test_dict[name] = probs
        
        # 保存 Test CSV
        csv_name = f"{name}_xgb.csv"
        save_to_csv(df_test, probs, out_dir, csv_name)
        
    plot_score_distributions(val_dict, test_dict, out_dir, model_name="XGBoost")
    return clf

# --- MLP (PyTorch) ---
class SimpleMLP(nn.Module):
    def __init__(self, in_dim, hidden_dim=128):
        super(SimpleMLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.net(x)

def run_mlp(X_train, y_train, X_val, y_val, df_val, test_data, out_dir, device='cuda'):
    print("\n=== Training MLP ===")
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    
    train_dataset = TensorDataset(torch.FloatTensor(X_train_scaled), torch.FloatTensor(y_train).unsqueeze(1))
    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
    
    model = SimpleMLP(in_dim=X_train.shape[1]).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.BCELoss()
    
    epochs = 30
    model.train()
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
    model.eval()
    
    # --- 验证集预测与保存 ---
    with torch.no_grad():
        X_val_tensor = torch.FloatTensor(X_val_scaled).to(device)
        val_probs = model(X_val_tensor).cpu().numpy().flatten()
    
    # 保存 Validation CSV (注意：这里保存的是df_val原始值，不是scaler处理后的)
    save_to_csv(df_val, val_probs, out_dir, "validation_set_mlp.csv")
        
    val_dict = {
        'sig': val_probs[y_val == 1],
        'bkg': val_probs[y_val == 0]
    }
    
    # --- 测试集预测与保存 ---
    test_dict = {}
    with torch.no_grad():
        for name, (X_test, df_test) in test_data.items():
            X_test_scaled = scaler.transform(X_test)
            X_tensor = torch.FloatTensor(X_test_scaled).to(device)
            probs = model(X_tensor).cpu().numpy().flatten()
            test_dict[name] = probs
            
            # 保存 Test CSV
            csv_name = f"{name}_mlp.csv"
            save_to_csv(df_test, probs, out_dir, csv_name)
            
    plot_score_distributions(val_dict, test_dict, out_dir, model_name="MLP")
    return model

# ==========================================
# Main Execution
# ==========================================
if __name__ == "__main__":
    # 路径配置
    sig_file = "/home/zhonghua/data/Dataset_Filted/filted_Monopole_1e10_merged_1000_70_new_dataset_strict_300_500.npz"
    pr_file  = "/home/zhonghua/data/Dataset_Filted/CosmicRay/Npz/Proton_1000_70_1e10_V03_dataset_strict_300_500.npz"
    
    test_files = {
        # "EF_Model": "/home/zhonghua/data/Dataset_Filted/CosmicRay/Npz/0818_filted_EF_Proton_4e13_1e15_dataset_strict.npz",
        # "QF_Model": "/home/zhonghua/data/Dataset_Filted/CosmicRay/Npz/0818_filted_QF_Proton_4e13_1e15_dataset_strict.npz",
        "Exp_2022": "/home/zhonghua/data/Dataset_Filted/Experiment/2022/all_combined_2022_dataset_1e10_for_diffmodels_strict_300_500.npz",
        # "QG_large": "/home/zhonghua/data/Dataset_Filted/CosmicRay/Npz/Proton_1000_70_1e10_V03_dataset_strict_300_500.npz",
        # "Monopole_large": "/home/zhonghua/data/Dataset_Filted/filted_Monopole_1e10_merged_1000_70_new_dataset_strict_300_500.npz",
    }
    
    # 特征列表
    feature_cols = ['R_mean', 'Eage','NuW2', 'rec_theta', 'rec_phi']
    output_dir = "/home/zhonghua/Filt_Event/model_validation/xgb_mlp_strict_300_500"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 设备配置
    
    device = "cpu" # 如果有GPU可改为 "cuda"
    
    # 1. 准备数据
    print("Preparing data...")
    # 注意：prepare_data 现在返回 df_val 和 test_data (包含完整df)
    X_train, y_train, X_val, y_val, df_val, test_data = prepare_data(
        sig_file, pr_file, test_files, feature_cols
    )
    print(f"Train size: {X_train.shape[0]}, Val size: {X_val.shape[0]}")
    
    # 2. 运行 XGBoost (会自动保存CSV)
    run_xgboost(X_train, y_train, X_val, y_val, df_val, test_data, output_dir)
    
    # # 3. 运行 MLP (会自动保存CSV)
    # run_mlp(X_train, y_train, X_val, y_val, df_val, test_data, output_dir, device=device)
    
    print("\nAll Done.")


Preparing data...
Loading: /home/zhonghua/data/Dataset_Filted/filted_Monopole_1e10_merged_1000_70_new_dataset_strict_300_500.npz
Loading: /home/zhonghua/data/Dataset_Filted/CosmicRay/Npz/Proton_1000_70_1e10_V03_dataset_strict_300_500.npz
Loading: /home/zhonghua/data/Dataset_Filted/Experiment/2022/all_combined_2022_dataset_1e10_for_diffmodels_strict_300_500.npz
Train size: 29845, Val size: 7462

=== Training XGBoost ===


/home/zhonghua/miniconda3/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [20:57:42] WARNING: /croot/xgboost-split_1749630910898/work/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Saving CSV: /home/zhonghua/Filt_Event/model_validation/xgb_mlp_strict_300_500/validation_set_xgb.csv
Saving CSV: /home/zhonghua/Filt_Event/model_validation/xgb_mlp_strict_300_500/Exp_2022_xgb.csv
Plot saved to /home/zhonghua/Filt_Event/model_validation/xgb_mlp_strict_300_500/XGBoost_dist.png

All Done.


In [ ]:
# 分析参数
import numpy as np
import pandas as pd
import json
import os
import uproot
# =====================================================
# 1. IO 工具
# =====================================================

def read_csv_df(path):
    if not os.path.exists(path):
        raise FileNotFoundError(path)
    df=pd.read_csv(path)
    df['weight']=np.ones(len(df))
    return df


def read_val_scores_new(path):
    if not os.path.exists(path):
        raise FileNotFoundError(path)
    data= pd.read_csv(path)
    sig_data=data[data['label']==43]
    bkg_data=data[data['label']==14]
    return sig_data, bkg_data


def give_weights(params_df):
    rootfile = "/home/zhonghua/data/exposure.root"
    with uproot.open(rootfile) as f:
        hWeight = f["hWeight"]
        edges_E = hWeight.axes[0].edges()  # Energy bin edges
        edges_Zen = hWeight.axes[1].edges()  # Zenith bin edges
        weights_2d = hWeight.values()

    def add_weights(trueE, rec_zenith, weights_2d):
        trueE = np.asarray(trueE, dtype=float)
        trueE=np.log10(trueE/1e3)
        rec_zenith=180/np.pi*rec_zenith
        bin_idx_E = np.digitize(trueE, edges_E) - 1 
        bin_idx_Zen = np.digitize(rec_zenith, edges_Zen) - 1
        
        return weights_2d[bin_idx_E, bin_idx_Zen] / 10 
    trueE = params_df['trueE'].values
    rec_zenith = params_df['theta'].values
    weights = add_weights(trueE, rec_zenith, weights_2d)
    params_df['weight'] = weights
    return params_df

def give_sig_weights(monopole_df):
    file = uproot.open("/home/zhonghua/Filt_Event/model_validation/exposure.root")
    h1 = file["h1"]
    h2 = file["h2"]
    # 获取直方图的所有数据
    values = h1.values()        # bin内容（纵坐标）
    h2_values = h2.values()        # bin内容（纵坐标）
    edges = h1.axis().edges()   # bin边界（横坐标）
    h2_edges = h2.axis().edges()   # bin边界（横坐标）
    centers = h1.axis().centers()  # bin中心
    h2_centers = h2.axis().centers()  # bin中心

    values = np.array(h1.values())
    edges = np.array(h1.axis().edges())
    centers = np.array(h1.axis().centers())
    S=np.pi * 1000**2
    parker=1e-15 * 1e4 
    nums=[]
    for i, theta in enumerate(centers):
        cosine_theta = np.cos(np.deg2rad(theta))
        area = S * cosine_theta
        exposure = values[i] * area
        num_i = exposure * parker
        nums.append(num_i)

    weights=[num_i/251000/h2_values[i]/86400 for i, num_i in enumerate(nums)]
    theta_degrees = monopole_df["theta"] * 180 / np.pi
    monopole_weights = []
    for theta in theta_degrees:
        if theta > max(centers) or theta < 0:
            weight = weights[-1] if theta > max(centers) else 0
        else:
            bin_id = np.digitize(theta, edges) - 1
            weight = weights[bin_id]
        monopole_weights.append(weight)

    monopole_df["weight"] = monopole_weights
    return monopole_df

# =====================================================
# 2. 配置
# =====================================================

base_dir = "/home/zhonghua/Filt_Event/model_validation/xgb_mlp"

EF_csv = f"{base_dir}/EF_Model_xgb.csv"
QF_csv = f"{base_dir}/QF_Model_xgb.csv"
QG_csv_large = f"{base_dir}/QG_large_xgb.csv"
val_npz = f"{base_dir}/validation_set_xgb.csv"
exp_2022_csv = f"{base_dir}/Exp_2022_xgb.csv"
exp_2022_csv = f"{base_dir}/Exp_2022_xgb.csv"
large_sig_csv = f"{base_dir}/Monopole_large_xgb.csv"

# =====================================================
# 3. 构建直方图
# =====================================================
# diff model EF+QF 
EF_scores  = read_csv_df(EF_csv)
QF_scores  = read_csv_df(QF_csv)
EF_scores = give_weights(EF_scores)
QF_scores = give_weights(QF_scores)
QG_scores_large  = read_csv_df(QG_csv_large)
QG_scores_large = give_weights(QG_scores_large)
# exp data
exp_2022_scores = read_csv_df(exp_2022_csv)
# val data
sig_scores, bkg_scores = read_val_scores_new(val_npz)
bkg_scores = give_weights(bkg_scores)
# large sig score
sig_scores = read_csv_df(large_sig_csv)
sig_scores = give_sig_weights(sig_scores)
# =====================================================
# 4. 绘图
# =====================================================
bins=30
# recE 
plt.hist(EF_scores["recE"], weights=EF_scores["weight"], bins=bins, histtype='step', label='EF', linestyle='--', density=True)
plt.hist(QF_scores["recE"], weights=QF_scores["weight"], bins=bins, histtype='step', label='QF', linestyle='--', density=True)
plt.hist(bkg_scores["recE"], weights=bkg_scores["weight"], bins=bins, histtype='step', label='QG', linestyle='--', density=True)
plt.hist(QG_scores_large["recE"], weights=QG_scores_large["weight"], bins=bins, histtype='step', label='QG large', density=True)
plt.hist(sig_scores["recE"], weights=sig_scores["weight"], bins=bins, histtype='stepfilled', alpha=0.3, label='SIG', density=True)
plt.hist(exp_2022_scores["recE"], weights=exp_2022_scores["weight"], bins=bins, histtype='stepfilled', alpha=0.3,label='Exp', linestyle='--', density=True)
plt.legend()
plt.xlabel("recE")
plt.show()
# rec_theta
plt.hist(EF_scores["rec_theta"], weights=EF_scores["weight"], bins=bins, histtype='step', label='EF', linestyle='--', density=True)
plt.hist(QF_scores["rec_theta"], weights=QF_scores["weight"], bins=bins, histtype='step', label='QF', linestyle='--', density=True)
plt.hist(bkg_scores["rec_theta"], weights=bkg_scores["weight"], bins=bins, histtype='step', label='QG', linestyle='--', density=True)
plt.hist(QG_scores_large["rec_theta"], weights=QG_scores_large["weight"], bins=bins, histtype='step', label='QG large', density=True)
plt.hist(sig_scores["rec_theta"], weights=sig_scores["weight"], bins=bins, histtype='stepfilled', alpha=0.3, label='SIG', density=True)
plt.hist(exp_2022_scores["rec_theta"], weights=exp_2022_scores["weight"], bins=bins, histtype='stepfilled', alpha=0.3,label='Exp', linestyle='--', density=True)
plt.legend()
plt.xlabel("rec_theta")
plt.show()
# Eage
plt.hist(EF_scores["Eage"], weights=EF_scores["weight"], bins=bins, histtype='step', label='EF', linestyle='--', density=True)
plt.hist(QF_scores["Eage"], weights=QF_scores["weight"], bins=bins, histtype='step', label='QF', linestyle='--', density=True)
plt.hist(bkg_scores["Eage"], weights=bkg_scores["weight"], bins=bins, histtype='step', label='QG', linestyle='--', density=True)
plt.hist(QG_scores_large["Eage"], weights=QG_scores_large["weight"], bins=bins, histtype='step', label='QG large', density=True)
plt.hist(sig_scores["Eage"], weights=sig_scores["weight"], bins=bins, histtype='stepfilled', alpha=0.3, label='SIG', density=True)
plt.hist(exp_2022_scores["Eage"], weights=exp_2022_scores["weight"], bins=bins, histtype='stepfilled', alpha=0.3,label='Exp', linestyle='--', density=True)
plt.legend()
plt.xlabel("Eage")
plt.show()
# R_mean
plt.hist(EF_scores["R_mean"], weights=EF_scores["weight"], bins=bins, histtype='step', label='EF', linestyle='--', density=True)
plt.hist(QF_scores["R_mean"], weights=QF_scores["weight"], bins=bins, histtype='step', label='QF', linestyle='--', density=True)
plt.hist(bkg_scores["R_mean"], weights=bkg_scores["weight"], bins=bins, histtype='step', label='QG', linestyle='--', density=True)
plt.hist(QG_scores_large["R_mean"], weights=QG_scores_large["weight"], bins=bins, histtype='step', label='QG large', density=True)
plt.hist(sig_scores["R_mean"], weights=sig_scores["weight"], bins=bins, histtype='stepfilled', alpha=0.3, label='SIG', density=True)
plt.hist(exp_2022_scores["R_mean"], weights=exp_2022_scores["weight"], bins=bins, histtype='stepfilled',alpha=0.3, label='Exp', linestyle='--', density=True)
plt.legend()
plt.xlabel("R_mean")
plt.show()
